In [20]:
from multiprocessing import Pool
from pathos.multiprocessing import Pool
from functools import partial
from concurrent.futures import ThreadPoolExecutor
import numpy as np


In [21]:
#TODO: loss of least square regression and binary logistic regression
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        return score

    def g(self,true,score):
        return 2 * (score - true)

    def h(self,true,score):
        return 2 * np.ones(score.shape[0])

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self,score):
        return 1 / (1 + np.exp(-score))

    def g(self,true,score):
        pred = self.pred(score)
        return pred - true

    def h(self,true,score):
        pred = self.pred(score)
        return pred * (1 - pred)

In [22]:
# TODO: class of a node on a tree
class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.
    
    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.
    
    A node on a tree can be either a leaf node or a non-leaf node.
    '''
    
    #TODO
    def __init__(
        self,
        split_feature = None,
        split_threshold = None,
        left_child = None,
        right_child = None,
        depth = None,
        weight = None
        ):
        
        # store essential information in every tree node
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.left_child = left_child
        self.right_child = right_child
        self.depth = depth
        self.weight = weight

        self.is_leaf = (self.left_child is None) and (self.right_child is None)
        
        
    

In [23]:
# TODO: class of single tree
class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''
    
    def __init__(self, n_threads = None, 
                 max_depth = 3, min_sample_split = 10,
                 lamda = 1, gamma = 0, rf = 0):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.int_member = 0

    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        #TODO
        
        self.root = self .construct_tree(train, g, h, max_depth = self.max_depth)
        return self

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        #TODO
        n_test = test.shape[0]
        result = []
        for i in range(n_test):
            each = test[i, :]
            node = self.root
            while not node.is_leaf:
                if each[node.split_feature] <= node.split_threshold and node.left_child is not None:
                    node = node.left_child
                else:
                    node = node.right_child
            result.append(node.weight)

        return np.array(result)

    def construct_tree(self, train, g, h, max_depth):
        '''
        Tree construction, which is recursively used to grow a tree.
        First we should check if we should stop further splitting.
        
        The stopping conditions include:
            1. tree reaches max_depth $d_{max}$
            2. The number of sample points at current node is less than min_sample_split, i.e., $n_{min}$
            3. gain <= 0
        '''
        #TODO
        # stopping condisiton 1
        weight = -np.sum(g) / (np.sum(h) + self.lamda)
        if max_depth == 0 or train.shape[0] < self.min_sample_split:
            return TreeNode(weight=weight)
        
        # # choose subset of features randomly
        # n_features = train.shape[1]
        # num_features_to_select = int(self.rf * n_features)
        # feature_indices = np.random.choice(n_features, num_features_to_select, replace=False)

        # find best splitting rule
        feature, threshold, gain= self.find_best_decision_rule(train, g, h)
        # feature = feature_indices[feature]      # project back

        # stopping condition 2
        if gain <= 0:
            return TreeNode(weight=weight)
        
        # divide data based on splitting rules
        left_indices = train[:,feature] <= threshold
        right_indices = train[:,feature] > threshold

        # recursive
        left_train, left_g, left_h = train[left_indices], g[left_indices], h[left_indices]
        right_train, right_g, right_h = train[right_indices], g[right_indices], h[right_indices]

        left_child = self.construct_tree(left_train, left_g, left_h, max_depth-1)
        right_child = self.construct_tree(right_train, right_g, right_h, max_depth-1)

        return TreeNode(split_feature = feature, split_threshold = threshold, 
                        left_child = left_child, right_child = right_child)

    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j, 
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train
        
        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        #TODO
        # train n * d
        # g: n * 1
        # h: n * 1
        n_features = train.shape[1]         
        feature = None
        threshold = None
        gain = float('-inf')
        
        num_features_to_select = int(self.rf * n_features)
        feature_indices = (
            np.random.choice(n_features, num_features_to_select, replace=False)
            if self.rf != 0 else np.arange(n_features)
            )
        
        # def process_feature(feature_idx):
        #     feature_column = train[:, feature_idx]
        #     return feature_idx, *self.find_threshold(g, h, feature_column)

        # multiprocessing
        if self.n_threads is not None and self.n_threads > 1:
            feature_columns = [train[:, idx] for idx in feature_indices]
            inputs = [(g, h, feature_columns[i]) for i in range(len(feature_columns))]
        
            with Pool(processes=self.n_threads) as pool:
                result = pool.starmap(self.find_threshold, inputs)
            # with ThreadPoolExecutor(max_workers=self.n_threads) as executor:
            #     results = executor.map(process_feature, idx)
            
            # print(result) [(return value of self.find_threshold), ()...()]
            # maybe confusing, just to align with the default return value so I use best_threshold, best_gain here instead of threshold, gain
            for feature_idx, (best_threshold, best_gain) in zip(feature_indices, result):
                if best_gain > gain:
                    gain = best_gain
                    threshold = best_threshold
                    feature = feature_idx


        else:
            for feature_idx in feature_indices:
                feature_column = train[:,feature_idx]
                best_threshold, best_gain = self.find_threshold(g, h, feature_column)
                if best_gain > gain:
                    gain = best_gain
                    threshold = best_threshold
                    feature = feature_idx
                

        return feature, threshold, gain
    
    def find_threshold(self, g, h, train):
        '''
        Given a particular feature $p_j$,   
        return the best split threshold $\tau_j$ together with the gain that is achieved.
        '''
        #TODO
        # sorting idx
        sorted_idx = np.argsort(train)          # eg 100 * 1
        sorted_train = train[sorted_idx]
        sorted_g = g[sorted_idx]
        sorted_h = h[sorted_idx]
        best_threshold = None
        best_gain = float('-inf')

        # gradient and hessian sum of all the nodes before splitting
        Gradient_sum = np.sum(sorted_g)
        Hessian_sum = np.sum(sorted_h)

        # initialize left and right nodes' sum of gradient and hessian
        # At the initial state, left node is empty and right node contains all the data
        G_left, G_right = 0, Gradient_sum
        H_left, H_right = 0, Hessian_sum

        for i in range(1, len(sorted_train)):
            # gradually move data in the right to left
            G_left += sorted_g[i-1]
            G_right -= sorted_g[i-1]
            H_left += sorted_h[i-1]
            H_right -= sorted_h[i-1]

            # when we encounter a different feature, we will do the split
            if sorted_train[i] != sorted_train[i - 1]:
                gain = 0.5 * ((G_left ** 2) / (H_left + self.lamda) + (G_right ** 2) / (H_right + self.lamda) - (Gradient_sum ** 2) / (Hessian_sum + self.lamda)) - self.gamma

                if gain > best_gain:
                    best_gain = gain
                    best_threshold = (sorted_train[i - 1] + sorted_train[i]) / 2  # middle point


        return [best_threshold, best_gain]

In [24]:
# TODO: class of Random Forest
import pandas as pd
class RF(object):
    '''
    Class of Random Forest
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        rf = 0.99, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees
        self.trees = []

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO

        # Ensure train and target are numpy arrays
        # if isinstance(train, pd.DataFrame):
        #     train = train.to_numpy()
        # if isinstance(target, pd.Series):
        #     target = target.to_numpy()
        


        n_samples = train.shape[0]
        # bootstrap with replacement
        for _ in range(self.num_trees):
            idx = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
            # check the idx if valid
            # assert idx.max() < n_samples, "Generated idx contrains out-of-bounds indices"

            sample_train, sample_target = train[idx], target[idx]
            
            # g, h
            pred = np.zeros(sample_target.shape[0])
            g, h = self.loss.g(sample_target, pred), self.loss.h(sample_target, pred)
            

            # train a new tree
            tree = Tree(n_threads = self.n_threads, rf = self.rf)
            tree.fit(sample_train, g, h)
            self.trees.append(tree)
        return self

    def predict(self, test):
        #TODO
        pred = np.array([tree.predict(test) for tree in self.trees])
        score = np.mean(pred, axis=0)
        return self.loss.pred(score)

In [25]:
# TODO: class of GBDT
class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth D_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        learning_rate: The learning rate eta of GBDT.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        learning_rate = None, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.trees = []

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        # initialize prediction value
        # pred = np.zeros(target.shape[0])
        pred = np.full(target.shape[0], np.mean(target)) # regression

        for _ in range(self.num_trees):
            g = self.learning_rate * self.loss.g(target, pred)
            h = self.learning_rate ** 2 * self.loss.h(target, pred)

            # train a new tree on gradient & hessian
            next_tree = Tree(n_threads=self.n_threads, rf = 0)
            next_tree.fit(train, g, h)
            self.trees.append(next_tree)
            pred = self.predict(train)          # consider all the previous trees
        # print(pred)
        return self

    def predict(self, test):
        #TODO
        score = 0
        for tree in self.trees:
            score += tree.predict(test)
        return self.loss.pred(score)

In [26]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# RMSE
def root_mean_square_error(pred, y):
    #TODO
    n = pred.shape[0]
    return np.sqrt(np.mean((pred - y) ** 2))

# precision
def accuracy(pred, y):
    #TODO
    return sum(pred == y)/ pred.shape[0]

In [27]:
# TODO: GBDT regression on boston house price dataset

# load data
import numpy as np
import pandas as pd

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]


# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/var/folders/j4/kk26gmps6pb74rjywjb8nmkc0000gn/T/ipykernel_75414/289120647.py:8: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


(506, 13) (506,) (354, 13) (354,) (152, 13) (152,)


In [28]:
# Problem 1.1
# RF for Boston house price dataset
model = RF(num_trees = 100, loss=leastsquare())
model.fit(X_train, y_train)

# results
predict_train = model.predict(X_train)
rmse_train = root_mean_square_error(predict_train, y_train)

predict_test = model.predict(X_test)
rmse_test = root_mean_square_error(predict_test, y_test)
print(f"RF rmse_train: {rmse_train}    RF rmse_test: {rmse_test}")

RF rmse_train: 3.4067206922906834    RF rmse_test: 4.190611271956216


In [29]:
# GBDT for Boston house price dataset
model = GBDT(num_trees = 100, learning_rate = 0.001, n_threads = 5, loss=leastsquare())
model.fit(X_train, y_train)

# results
predict_train = model.predict(X_train)
rmse_train = root_mean_square_error(predict_train, y_train)

predict_test = model.predict(X_test)
rmse_test = root_mean_square_error(predict_test, y_test)
print(f"GBDT rmse_train: {rmse_train}    GBDT rmse_test: {rmse_test}")

GBDT rmse_train: 1.7378787449656554    GBDT rmse_test: 3.4593227483389075


In [30]:
# Problem 2.6.4
# least square for Boston house price dataset
# from sklearn.linear_model import LinearRegression
# linear_model = LinearRegression()
# linear_model.fit(X_train, y_train)
W = np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train)), np.matmul(X_train.T, y_train))

# results
predict_train = np.matmul(X_train, W)
predict_test = np.matmul(X_test, W)
linear_rmse_train = root_mean_square_error(predict_train, y_train)
linear_rmse_test = root_mean_square_error(predict_test, y_test)
print(f"Least Square rmse_train: {linear_rmse_train}    Least Square rmse_test: {linear_rmse_test}")


Least Square rmse_train: 4.820626531838222    Least Square rmse_test: 5.209217510530889


In [31]:
# Problem 2.6.4
# ridge regression for Boston house price dataset
W = np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train) + 0.5), np.matmul(X_train.T, y_train))

# results
predict_train = np.matmul(X_train, W)
predict_test = np.matmul(X_test, W)
ridge_rmse_train = root_mean_square_error(predict_train, y_train)
ridge_rmse_test = root_mean_square_error(predict_test, y_test)
print(f"Ridge regression rmse_train: {ridge_rmse_train}    Ridge Regression rmse_test: {ridge_rmse_test}")

Ridge regression rmse_train: 4.822434482543473    Ridge Regression rmse_test: 5.186569984437072


In [32]:
# TODO: GBDT classification on credit-g dataset

# load data
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))
# X_train.head()

# Way 1
# preprocess the data
"""Since my model cannot deal with non-numeric data, need to preprocess the dataset
Here use a mapping to change the non-numeric data into numeric ones"""
# def prepreprocess_data(X):
#     mappings = {}
#     non_numeric_cols = X.select_dtypes(exclude='number').columns

#     for col in non_numeric_cols:
#         # generate mapping rule for each column
#         unique = X[col].unique()
#         mappings[col] = {each: i / len(unique) - 0.5 for i, each in enumerate(unique)}
#         if col in mappings:
#             X[col] = X[col].map(mappings[col])
#     return X, mappings

# Way2
from sklearn.preprocessing import LabelEncoder
def prepreprocess_data(dataframe):
    """Process the input dataframe by encoding categorical variables and
    ensuring numerical columns are integers."""
    # categorical columns
    categorical_cols = [
        'checking_status', 'credit_history', 'purpose', 'savings_status',
        'employment', 'personal_status', 'other_parties', 'property_magnitude',
        'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker'
    ]
    
    # Encode categorical columns
    label_encoders = {}
    for col in categorical_cols:
        labelencoder = LabelEncoder()
        dataframe[col] = labelencoder.fit_transform(dataframe[col])
        # label_encoders[col] = labelencoder  # Store encoder for potential inverse transformations later
    
    # Ensure numerical columns are integers
    numerical_cols = dataframe.select_dtypes(include=['int64', 'float64']).columns
    for col in numerical_cols:
        dataframe[col] = dataframe[col].astype(int)
    
    return dataframe, label_encoders

X, label_encoders = prepreprocess_data(X)


# X, mappings = prepreprocess_data(X)
# print(X)
X = X.values
# print(X)

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1000, 20) (1000,) (700, 20) (700,) (300, 20) (300,)


In [33]:
# explore the data
# print("First 5 rows of X:")
# print(X[:5])
# print("First 5 values of y:")
# print(y[:5])
# print(X.columns)
# for col in X.columns:
#     print(f"{col} | number of unique values: {X[col].nunique()}")
# print('-----------------------------------------------')
# print(y)

In [34]:
# Problem 1.2
# RF for Credit-g dataset
model = RF(num_trees=100, loss=logistic())
model.fit(X_train, y_train)

# results
predict_train = model.predict(X_train)>0.5
accuracy_train = accuracy(predict_train, y_train)

predict_test = model.predict(X_test)>0.5
accuracy_test = accuracy(predict_test, y_test)

print(f"RF train accuracy: {accuracy_train}    RF test accuracy: {accuracy_test}")


RF train accuracy: 0.76    RF test accuracy: 0.7466666666666667


In [35]:
# Problem 2.6.5
# GBDT for Credit-g dataset
model = GBDT(num_trees = 100, learning_rate = 0.003, n_threads = 5, loss=logistic())
model.fit(X_train, y_train)

# results
predict_train = model.predict(X_train)>0.5
accuracy_train = accuracy(predict_train, y_train)

predict_test = model.predict(X_test)>0.5
accuracy_test = accuracy(predict_test, y_test)

print(f"GBDT train accuracy: {accuracy_train}    GBDT test accuracy: {accuracy_test}")

GBDT train accuracy: 0.7585714285714286    GBDT test accuracy: 0.7266666666666667


In [36]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(569, 30) (569,) (398, 30) (398,) (171, 30) (171,)


In [37]:
# Problem 1.2
# breast cancer with RF
model = RF(num_trees=100, loss=logistic())
model.fit(X_train, y_train)

threshold = 0.5
predict_train = model.predict(X_train) > threshold
accuracy_train = accuracy(predict_train, y_train)

predict_test = model.predict(X_test) > threshold
accuracy_test = accuracy(predict_test, y_test)

print(f"RF train accuracy: {accuracy_train}    RF test accuracy: {accuracy_test}")


RF train accuracy: 0.9849246231155779    RF test accuracy: 0.9649122807017544


In [38]:
# Problem 2.6.6
# breast cancer with GDBT
model = GBDT(num_trees = 100, learning_rate = 0.04, n_threads = 5, loss=logistic())
model.fit(X_train, y_train)

# results
predict_train = model.predict(X_train)>0.5
accuracy_train = accuracy(predict_train, y_train)

predict_test = model.predict(X_test)>0.5
accuracy_test = accuracy(predict_test, y_test)

print(f"GBDT train accuracy: {accuracy_train}    GBDT test accuracy: {accuracy_test}")

GBDT train accuracy: 0.964824120603015    GBDT test accuracy: 0.9239766081871345
